## Advanced Example - Social Network Analysis

In [1]:
from pydantic import BaseModel
from typing import List, Optional, Dict, Any
from datetime import datetime
from pydapter.core import Adaptable
from pydapter.extras.neo4j_ import Neo4jAdapter
from neo4j import GraphDatabase
import random

# Neo4j connection settings
NEO4J_URI = "bolt://localhost:7687"
NEO4J_AUTH = ("neo4j", "password")


# Define our models
class User(BaseModel, Adaptable):
    id: str
    username: str
    full_name: Optional[str] = None
    email: Optional[str] = None
    location: Optional[str] = None
    joined_date: Optional[str] = None


class Post(BaseModel, Adaptable):
    id: str
    content: str
    created_at: str
    likes: int = 0
    user_id: str  # Author of the post


# Register adapters
User.register_adapter(Neo4jAdapter)
Post.register_adapter(Neo4jAdapter)


# Helper function to create Neo4j driver
def get_driver():
    return GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)


# Initialize the database with schema and constraints
def initialize_database():
    driver = get_driver()

    with driver.session() as session:
        # Create constraints for uniqueness
        session.run(
            "CREATE CONSTRAINT IF NOT EXISTS FOR (u:User) REQUIRE u.id IS UNIQUE"
        )
        session.run(
            "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Post) REQUIRE p.id IS UNIQUE"
        )

    driver.close()
    print("Database initialized with constraints")


# Create relationships between users (follows) and between users and posts
def create_relationships(users, posts):
    driver = get_driver()

    with driver.session() as session:
        # Connect users with their posts
        print("\nConnecting users with their posts...")
        for post in posts:
            session.run(
                """
                MATCH (u:User {id: $user_id})
                MATCH (p:Post {id: $post_id})
                MERGE (u)-[:POSTED]->(p)
                """,
                user_id=post.user_id,
                post_id=post.id,
            )
            print(f"Connected user {post.user_id} with post {post.id}")

        # Create random follow relationships between users
        print("\nCreating follow relationships...")
        user_ids = [user.id for user in users]

        for user_id in user_ids:
            # Each user follows a random subset of other users
            for other_id in user_ids:
                if (
                    user_id != other_id and random.random() < 0.3
                ):  # 30% chance to follow
                    session.run(
                        """
                        MATCH (u1:User {id: $user_id})
                        MATCH (u2:User {id: $other_id})
                        MERGE (u1)-[:FOLLOWS]->(u2)
                        """,
                        user_id=user_id,
                        other_id=other_id,
                    )
                    print(f"User {user_id} follows User {other_id}")

        # Create some likes on posts
        print("\nCreating likes on posts...")
        for user_id in user_ids:
            for post in posts:
                # Users don't like their own posts, and random chance to like others
                if (
                    post.user_id != user_id and random.random() < 0.4
                ):  # 40% chance to like
                    session.run(
                        """
                        MATCH (u:User {id: $user_id})
                        MATCH (p:Post {id: $post_id})
                        MERGE (u)-[:LIKES]->(p)
                        """,
                        user_id=user_id,
                        post_id=post.id,
                    )

                    # Also update the likes count on the post
                    session.run(
                        """
                        MATCH (p:Post {id: $post_id})
                        SET p.likes = p.likes + 1
                        """,
                        post_id=post.id,
                    )

                    print(f"User {user_id} likes Post {post.id}")

    driver.close()


# Populate the database with users and posts
def populate_database():
    # Create some users
    users = [
        User(
            id="u1",
            username="alice_wonder",
            full_name="Alice Wonderland",
            email="alice@example.com",
            location="New York",
            joined_date=datetime(2022, 1, 15).isoformat(),
        ),
        User(
            id="u2",
            username="bob_builder",
            full_name="Bob Builder",
            email="bob@example.com",
            location="San Francisco",
            joined_date=datetime(2022, 2, 20).isoformat(),
        ),
        User(
            id="u3",
            username="charlie_brown",
            full_name="Charlie Brown",
            email="charlie@example.com",
            location="Chicago",
            joined_date=datetime(2022, 3, 10).isoformat(),
        ),
        User(
            id="u4",
            username="david_jones",
            full_name="David Jones",
            email="david@example.com",
            location="Miami",
            joined_date=datetime(2022, 4, 5).isoformat(),
        ),
        User(
            id="u5",
            username="emma_stone",
            full_name="Emma Stone",
            email="emma@example.com",
            location="Los Angeles",
            joined_date=datetime(2022, 5, 1).isoformat(),
        ),
    ]

    # Create some posts
    posts = [
        Post(
            id="p1",
            content="Just learned about Neo4j and graph databases!",
            created_at=datetime(2023, 1, 5).isoformat(),
            user_id="u1",
        ),
        Post(
            id="p2",
            content="Excited to start my new project with Python",
            created_at=datetime(2023, 1, 10).isoformat(),
            user_id="u1",
        ),
        Post(
            id="p3",
            content="San Francisco has the best views!",
            created_at=datetime(2023, 1, 8).isoformat(),
            user_id="u2",
        ),
        Post(
            id="p4",
            content="Working on a new recommendation algorithm",
            created_at=datetime(2023, 1, 12).isoformat(),
            user_id="u3",
        ),
        Post(
            id="p5",
            content="Just finished reading a great book about AI",
            created_at=datetime(2023, 1, 15).isoformat(),
            user_id="u3",
        ),
        Post(
            id="p6",
            content="Miami sunsets are unbeatable!",
            created_at=datetime(2023, 1, 14).isoformat(),
            user_id="u4",
        ),
        Post(
            id="p7",
            content="Excited about new movie roles coming up",
            created_at=datetime(2023, 1, 18).isoformat(),
            user_id="u5",
        ),
    ]

    # Store users in Neo4j
    print("Storing users...")
    for user in users:
        user.adapt_to(
            obj_key="neo4j", url=NEO4J_URI, auth=NEO4J_AUTH, label="User", merge_on="id"
        )

    # Store posts in Neo4j
    print("\nStoring posts...")
    for post in posts:
        post.adapt_to(
            obj_key="neo4j", url=NEO4J_URI, auth=NEO4J_AUTH, label="Post", merge_on="id"
        )

    # Create relationships
    create_relationships(users, posts)

    print("Database populated with sample data")


# Function to get a user's feed (posts from users they follow)
def get_user_feed(user_id):
    """Get posts from users that this user follows"""
    driver = get_driver()

    feed_posts = []

    with driver.session() as session:
        result = session.run(
            """
            MATCH (u:User {id: $user_id})-[:FOLLOWS]->(friend:User)-[:POSTED]->(p:Post)
            RETURN p, friend.username AS author
            ORDER BY p.created_at DESC
            LIMIT 10
            """,
            user_id=user_id,
        )

        for record in result:
            post_data = dict(record["p"].items())
            post = Post(**post_data)
            author = record["author"]
            feed_posts.append((post, author))

    driver.close()
    return feed_posts


# Function to get recommended users to follow
def get_follow_recommendations(user_id):
    """Recommend users to follow based on mutual connections"""
    driver = get_driver()

    recommended_users = []

    with driver.session() as session:
        # Find users who are followed by people the user follows,
        # but the user doesn't follow yet
        result = session.run(
            """
            MATCH (user:User {id: $user_id})-[:FOLLOWS]->(mutual:User)-[:FOLLOWS]->(recommended:User)
            WHERE NOT (user)-[:FOLLOWS]->(recommended)
            AND user.id <> recommended.id
            WITH recommended, count(mutual) AS mutualCount
            ORDER BY mutualCount DESC
            LIMIT 5
            RETURN recommended
            """,
            user_id=user_id,
        )

        for record in result:
            user_data = dict(record["recommended"].items())
            user = User(**user_data)
            recommended_users.append(user)

    driver.close()
    return recommended_users


# Function to get popular posts
def get_popular_posts():
    """Get posts with the most likes"""
    driver = get_driver()

    popular_posts = []

    with driver.session() as session:
        result = session.run(
            """
            MATCH (p:Post)
            WITH p, p.likes AS likes
            ORDER BY likes DESC
            LIMIT 5
            MATCH (author:User)-[:POSTED]->(p)
            RETURN p, author.username AS author
            """
        )

        for record in result:
            post_data = dict(record["p"].items())
            post = Post(**post_data)
            author = record["author"]
            popular_posts.append((post, author))

    driver.close()
    return popular_posts


# Main function to demo the social network
def main():
    # Initialize and populate the database
    initialize_database()
    populate_database()

    # Get user feed for Alice
    print("\nAlice's feed (posts from people she follows):")
    feed = get_user_feed("u1")
    for post, author in feed:
        print(f"@{author}: {post.content}")
        print(f"  Likes: {post.likes} | Posted: {post.created_at}")

    # Get recommended users for Bob to follow
    print("\nRecommended users for Bob to follow:")
    recommendations = get_follow_recommendations("u2")
    for user in recommendations:
        print(f"  - {user.full_name} (@{user.username}) from {user.location}")

    # Get popular posts
    print("\nPopular posts across the network:")
    popular = get_popular_posts()
    for i, (post, author) in enumerate(popular):
        print(f"{i + 1}. @{author}: {post.content}")
        print(f"   Likes: {post.likes}")


main()

Database initialized with constraints
Storing users...

Storing posts...

Connecting users with their posts...
Connected user u1 with post p1
Connected user u1 with post p2
Connected user u2 with post p3
Connected user u3 with post p4
Connected user u3 with post p5
Connected user u4 with post p6
Connected user u5 with post p7

Creating follow relationships...
User u1 follows User u2
User u1 follows User u3
User u2 follows User u4
User u3 follows User u2

Creating likes on posts...
User u1 likes Post p6
User u1 likes Post p7
User u2 likes Post p1
User u2 likes Post p2
User u3 likes Post p6
User u3 likes Post p7
User u4 likes Post p4
User u5 likes Post p1
User u5 likes Post p6
Database populated with sample data

Alice's feed (posts from people she follows):
@charlie_brown: Just finished reading a great book about AI
  Likes: 0 | Posted: 2023-01-15T00:00:00
@david_jones: Miami sunsets are unbeatable!
  Likes: 3 | Posted: 2023-01-14T00:00:00
@charlie_brown: Working on a new recommendation